In [ ]:
! pip install torch torchvision torchaudio pypdf

In [ ]:
from transformers import pipeline
from pypdf import PdfReader

In [ ]:
summarizer = pipeline(task="summarization", model="facebook/bart-large-cnn")

##Trying out first with text data

In [ ]:
reader = PdfReader("/content/sample_data/data.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()
text

In [ ]:
results = summarizer(text, max_length=100, do_sample=False)
results

In [ ]:
results[0]["summary_text"]

##Further Modifications to include pdf that has images as well as links

In [ ]:
import re

In [ ]:

# Function to preprocess text from PDF
def preprocess_text(text):
    # Remove any URLs
    text = re.sub(r'http\S+', '', text)
    # Remove any image captions or descriptions
    text = re.sub(r'\[IMAGE:.*?\]', '', text)
    # Remove any remaining special characters or noise
    text = re.sub(r'\n', ' ', text)  # Remove newline characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    return text



In [ ]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    return text



In [ ]:

# Path to the PDF file
pdf_path = "/content/sample_data/Resume_QA.pdf" #Trying out with pdf that has links

# Extract text from PDF
text = extract_text_from_pdf(pdf_path)

# Preprocess the text
text = preprocess_text(text)

# Summarize the preprocessed text
results = summarizer(text, max_length=100, do_sample=False)

# Display the summary
summary = results[0]["summary_text"]
print(summary)


In [ ]:
summary

In [ ]:

# Path to the PDF file
pdf_path = "/content/sample_data/data2.pdf" #Trying out with pdf that has images

# Extract text from PDF
text = extract_text_from_pdf(pdf_path)

# Preprocess the text
text = preprocess_text(text)

# Summarize the preprocessed text
results = summarizer(text, max_length=100, do_sample=False)

# Display the summary
summary = results[0]["summary_text"]
print(summary)


In [ ]:
summary